In [1]:
spark

In [2]:
# importing module
import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

In [3]:
# creating sparksession and giving an app name
spark = SparkSession.builder.appName('sparkdf').getOrCreate()

In [11]:
# create DataFrame
df=spark.read.option(
  "header",True).option('inferSchema',True).csv("Cricket_data.csv")

In [13]:
# Display schema
df.printSchema() 

root
 |-- Team: string (nullable = true)
 |-- Player_ID: integer (nullable = true)
 |-- Player: string (nullable = true)
 |-- Matches: integer (nullable = true)
 |-- Runs: integer (nullable = true)
 |-- Wickets: integer (nullable = true)
 |-- Hundreds: integer (nullable = true)
 |-- Fifties: integer (nullable = true)
 |-- Speciality: string (nullable = true)



In [14]:
df.write.option("header", True) \
        .partitionBy("Team") \
        .mode("overwrite") \
        .csv("Team")

In [15]:
# From above DataFrame, we will be using 
# Team and Speciality as a partition key 
# for our examples below.
# partitionBy()
df.write.option("header", True) \
        .partitionBy("Team", "Speciality") \
        .mode("overwrite") \
        .csv("Team-Speciality")

In [20]:
#Use repartition() and partitionBy() together
df.repartition(2).write.option("header",True) \
        .partitionBy("Team") \
        .mode("overwrite") \
        .csv("team-more")

In [22]:
# partitionBy() control number of partitions
df.write.option("header",True) \
        .option("maxRecordsPerFile", 2) \
        .partitionBy("Team",'Speciality') \
        .mode("overwrite") \
        .csv("Team")

In [21]:
dfSinglePart=spark.read.option("header",True) \
            .csv("Team/Team=Aus")
dfSinglePart.printSchema()
dfSinglePart.show()

root
 |-- Player_ID: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- Matches: string (nullable = true)
 |-- Runs: string (nullable = true)
 |-- Wickets: string (nullable = true)
 |-- Hundreds: string (nullable = true)
 |-- Fifties: string (nullable = true)
 |-- Speciality: string (nullable = true)

+---------+--------------+-------+----+-------+--------+-------+----------+
|Player_ID|        Player|Matches|Runs|Wickets|Hundreds|Fifties|Speciality|
+---------+--------------+-------+----+-------+--------+-------+----------+
|    49991|Ricky Pointing|    293|7758|     36|      32|     28|       Bat|
|   548172|  Mathew Hyden|    327|1427|     28|      37|     78|       Bat|
|   519952| Glenn McGrath|    254| 867|    327|       0|      0|       Bow|
|    16116|     Brett Lee|    416|1329|    266|       0|      0|       Bow|
+---------+--------------+-------+----+-------+--------+-------+----------+



In [24]:
dfSinglePart=spark.read.option("header",True) \
            .csv("Team/Team=Aus/Speciality=Bat")
dfSinglePart.printSchema()
dfSinglePart.show()

root
 |-- Player_ID: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- Matches: string (nullable = true)
 |-- Runs: string (nullable = true)
 |-- Wickets: string (nullable = true)
 |-- Hundreds: string (nullable = true)
 |-- Fifties: string (nullable = true)

+---------+--------------+-------+----+-------+--------+-------+
|Player_ID|        Player|Matches|Runs|Wickets|Hundreds|Fifties|
+---------+--------------+-------+----+-------+--------+-------+
|    49991|Ricky Pointing|    293|7758|     36|      32|     28|
|   548172|  Mathew Hyden|    327|1427|     28|      37|     78|
+---------+--------------+-------+----+-------+--------+-------+



In [25]:
#Missing Values
from pyspark.sql import SparkSession

In [26]:
spark = SparkSession.builder.appName('Practice').getOrCreate()

In [27]:
df=spark.read.option('header','True').csv('test2.csv',inferSchema=True)

In [28]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [54]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
df=df.withColumn("Salary",col("Salary").cast(IntegerType()))

In [55]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: integer (nullable = true)



In [29]:
df.show()

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
| Krish|  31|        10| 30000|
|  John|  30|         8| 25000|
| Surya|  29|         4| 20000|
|  Paul|  24|         3| 20000|
|Harsha|  21|         1| 15000|
| Subha|  23|         2| 18000|
|Mahesh|null|      null| 40000|
|  null|  34|        10| 38000|
|  null|  36|      null|  null|
+------+----+----------+------+



In [30]:
df.columns

['Name', 'age', 'Experience', 'Salary']

In [31]:
# Drop Columns
df.drop('age').show()

+------+----------+------+
|  Name|Experience|Salary|
+------+----------+------+
| Krish|        10| 30000|
|  John|         8| 25000|
| Surya|         4| 20000|
|  Paul|         3| 20000|
|Harsha|         1| 15000|
| Subha|         2| 18000|
|Mahesh|      null| 40000|
|  null|        10| 38000|
|  null|      null|  null|
+------+----------+------+



In [32]:
#Drop null values
df.na.drop().show()                      #Drops null rows

+------+---+----------+------+
|  Name|age|Experience|Salary|
+------+---+----------+------+
| Krish| 31|        10| 30000|
|  John| 30|         8| 25000|
| Surya| 29|         4| 20000|
|  Paul| 24|         3| 20000|
|Harsha| 21|         1| 15000|
| Subha| 23|         2| 18000|
+------+---+----------+------+



In [33]:
df.na.drop(how='all').show()              # will drop row which will have all column values as null

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
| Krish|  31|        10| 30000|
|  John|  30|         8| 25000|
| Surya|  29|         4| 20000|
|  Paul|  24|         3| 20000|
|Harsha|  21|         1| 15000|
| Subha|  23|         2| 18000|
|Mahesh|null|      null| 40000|
|  null|  34|        10| 38000|
|  null|  36|      null|  null|
+------+----+----------+------+



In [34]:
df.na.drop(how='any').show()         # drop any row which will consist atleast one null value 

+------+---+----------+------+
|  Name|age|Experience|Salary|
+------+---+----------+------+
| Krish| 31|        10| 30000|
|  John| 30|         8| 25000|
| Surya| 29|         4| 20000|
|  Paul| 24|         3| 20000|
|Harsha| 21|         1| 15000|
| Subha| 23|         2| 18000|
+------+---+----------+------+



In [35]:
df.na.drop(how='any',thresh=2).show()    # will keep row which will have atleast 2 no_null values 

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
| Krish|  31|        10| 30000|
|  John|  30|         8| 25000|
| Surya|  29|         4| 20000|
|  Paul|  24|         3| 20000|
|Harsha|  21|         1| 15000|
| Subha|  23|         2| 18000|
|Mahesh|null|      null| 40000|
|  null|  34|        10| 38000|
|  null|  36|      null|  null|
+------+----+----------+------+



In [36]:
df.na.drop(how='any',thresh=3).show()

+------+---+----------+------+
|  Name|age|Experience|Salary|
+------+---+----------+------+
| Krish| 31|        10| 30000|
|  John| 30|         8| 25000|
| Surya| 29|         4| 20000|
|  Paul| 24|         3| 20000|
|Harsha| 21|         1| 15000|
| Subha| 23|         2| 18000|
|  null| 34|        10| 38000|
|  null| 36|      null|  null|
+------+---+----------+------+



In [37]:
df.na.drop(how='any',subset='age').show()           #drop rows which have null value in specified column in subset

+------+---+----------+------+
|  Name|age|Experience|Salary|
+------+---+----------+------+
| Krish| 31|        10| 30000|
|  John| 30|         8| 25000|
| Surya| 29|         4| 20000|
|  Paul| 24|         3| 20000|
|Harsha| 21|         1| 15000|
| Subha| 23|         2| 18000|
|  null| 34|        10| 38000|
|  null| 36|      null|  null|
+------+---+----------+------+



In [56]:
# Fill Values
df.na.fill('Filler',['age','Salary']).show()

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
| Krish|  31|        10| 30000|
|  John|  30|         8| 25000|
| Surya|  29|         4| 20000|
|  Paul|  24|         3| 20000|
|Harsha|  21|         1| 15000|
| Subha|  23|         2| 18000|
|Mahesh|null|      null| 40000|
|  null|  34|        10| 38000|
|  null|  36|      null|  null|
+------+----+----------+------+



In [50]:
df.show()

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
| Krish|  31|        10| 30000|
|  John|  30|         8| 25000|
| Surya|  29|         4| 20000|
|  Paul|  24|         3| 20000|
|Harsha|  21|         1| 15000|
| Subha|  23|         2| 18000|
|Mahesh|null|      null| 40000|
|  null|  34|        10| 38000|
|  null|  36|      null|  null|
+------+----+----------+------+



In [51]:
df.dtypes

[('Name', 'string'),
 ('age', 'int'),
 ('Experience', 'string'),
 ('Salary', 'string')]

In [57]:
df.na.fill(0).show()

+------+---+----------+------+
|  Name|age|Experience|Salary|
+------+---+----------+------+
| Krish| 31|        10| 30000|
|  John| 30|         8| 25000|
| Surya| 29|         4| 20000|
|  Paul| 24|         3| 20000|
|Harsha| 21|         1| 15000|
| Subha| 23|         2| 18000|
|Mahesh|  0|      null| 40000|
|  null| 34|        10| 38000|
|  null| 36|      null|     0|
+------+---+----------+------+

